# **ICT3464 - Ruteo de Vehículos**

Profesor: Homero Larraín Izquierdo (hlarraii@uc.cl)

Ayudantes: Diego San Martín (dsm@uc.cl), Benjamín Rojas (bgrojas@uc.cl)

In [1]:
%pip install osmnx
%pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import folium
import pickle
import os

## Parte 0: Algunas funciones útiles

#### Función para calcular el costo de una solución

In [3]:
def generar_matriz_de_costos(Distancias, C_V, C_F):

    n = len(Distancias)

    Costos = np.zeros(shape = (n, n))

    for i in range(len(Distancias)):
        for j in range(len(Distancias[i])):
            if i!=j:
                if i==0 or j==0: # A los arcos que salen y entran del depot, le agregamos la mitad del costo fijo
                    Costos[i, j] = C_F/2 + C_V * Distancias[i][j]/1000
                else: # En caso contrario, solo ponderamos la distancia por el costo variable
                    Costos[i, j] = C_V * Distancias[i][j]/1000
            else:
                Costos[i, j] = 0

    return Costos

In [4]:
n = 10

#### Función para verificar que una solución es factible

In [5]:
def es_factible(Rutas, Demandas, Q, n):

    # Verificamos la restricción de capacidad:
     # Calculamos la carga de cada ruta por separado
    for ruta in Rutas:
        demanda_ruta = 0
        for nodo in ruta:
            demanda_ruta += Demandas[nodo]

        # Si alguna ruta tiene una carga mayor a la capacidad, se devuelve mensaje de error
        if demanda_ruta > Q:
            return 'Error. La ruta es infactible por capacidad.'


    # Verificamos que se estén visitando todos los clientes
     # Generamos una lista con todos los clientes
    lista_clientes = []
    for cliente in range(1, n):
        lista_clientes.append(cliente)

    # Eliminamos cada uno de los clientes visitados
    for ruta in Rutas:
        for nodo in ruta:
            if nodo != 0:
                lista_clientes.remove(nodo)

    if len(lista_clientes) > 0:
        return 'Error. La ruta es infactible. Hay clientes que no han sido visitados.'
    else: # Si todas las rutas tienen carga inferior a la capacidad, se devuelve mensaje de factibilidad
        return 'La ruta es factible.'

## Parte I-1: Matriz de costos, importar datos del problema y visualización

#### Función para generar una matriz de costos a partir de la matriz de distancias, costos variables y costos fijos

In [6]:
def generar_matriz_de_costos(Distancias, C_V, C_F):

    n = len(Distancias)

    Costos = np.zeros(shape = (n, n))

    for i in range(len(Distancias)):
        for j in range(len(Distancias[i])):
            if i!=j:
                if i==0 or j==0: # A los arcos que salen y entran del depot, le agregamos la mitad del costo fijo
                    Costos[i, j] = C_F/2 + C_V * Distancias[i][j]/1000
                else: # En caso contrario, solo ponderamos la distancia por el costo variable
                    Costos[i, j] = C_V * Distancias[i][j]/1000
            else:
                Costos[i, j] = 0

    return Costos

In [7]:
# Tamaño de la instancia
n = 10

In [8]:
file = "Talca-n" + str(n) + ".pkl"
with open(file, 'rb') as file:
    coordenadas = pickle.load(file) # Latitud, longidut de cada nodo
    caminos = pickle.load(file) # Caminos mínimos entre cada par de nodos. Lo usaremos más adelante para la visualización de las rutas en osmnx/folium.
    distancias = pickle.load(file) # Distancias entre cada par de nodos (m)
    demandas = pickle.load(file) # Demandas de los clientes. Para el nodo depot la demanda es 0. (u)
    capacidad = pickle.load(file) # Capacidad de los vehículos (u)
    c_v = pickle.load(file) # Costos variables ($/km)
    c_f = pickle.load(file) # Costos fijos ($/veh.)

# Creamos la matriz de costos
costos = generar_matriz_de_costos(distancias, c_v, c_f)

In [9]:
print(costos)
print('')
print(demandas)
print('')
print(capacidad)

[[   0.  2371.1 1673.  3157.1 2140.1 1602.2 1262.  2417.  2389.7 2441.3]
 [2602.1    0.  1372.5 2384.4 1352.4 1352.1 1254.  1768.2  772.2 1406.7]
 [1673.  1372.5    0.  3097.2 2065.2  336.  1677.9 2481.3 1574.1 2161.8]
 [3228.2 2366.4 3094.5    0.  1114.5 3074.1 2025.9 1032.  1665.3 1098.9]
 [2192.3 1290.6 2003.4 1238.7    0.  1983.   963.6  665.4  773.1  718.8]
 [1602.2 1344.   336.  3068.7 2036.7    0.  1607.1 2452.8 1521.3 2133.3]
 [1347.8 1247.7 1630.2 2058.  1036.5 1559.7    0.  1317.9 1127.4 1179. ]
 [2571.5 1905.9 2619.   947.7  770.1 2598.6 1435.5    0.  1530.3 1151.4]
 [2453.9  744.9 1568.7 1688.4  768.3 1548.6 1105.8 1432.8    0.   695.1]
 [2562.2 1427.4 2183.1 1066.5  695.7 2162.7 1214.1 1160.7  726.6    0. ]]

[0 4 1 2 1 5 4 1 1 5]

5


#### Generemos un mapa interactivo

In [10]:
# G es el grafo de Talca
place_name = 'Talca, Chile'

G = ox.graph_from_place(place_name, network_type='drive')

# Generamos un mapa en folium centrado en el Coliseo
m = folium.Map(location=[-35.426527777778, -71.666055555556], zoom_start=10, tiles='OpenStreetMap')

# Inicializamos un contador de nodos
i = 0

# Agregamos cada nodo en el mapa
for coord in coordenadas:
    icon = folium.Icon(icon="fa-solid fa-user",
                       prefix='fa',
                       color='red')
    folium.Marker(location=[coord[0], coord[1]],
                  popup=folium.Popup(f'Nodo {i}', max_width=75),
                  icon=icon).add_to(m)
    i += 1

#display(m)

## Parte I-2: Heurística constructiva

#### Heurística un cliente, un vehículo

In [11]:
def un_cliente_un_vehiculo(Costos, Demandas, Capacidad):

    n_clientes = len(Demandas) - 1
    rutas = []
    costo_total = 0

    for i in range(1, n_clientes + 1):
      costo = Costos[0][i] + Costos[i][0]
      costo_total += costo
      ruta=[costo,0,i,0]
      rutas.append(ruta)
    return rutas, costo_total

In [12]:
rutas = un_cliente_un_vehiculo(costos, demandas, capacidad)
print('Costo    Ruta')
for i in rutas[0]:
  print(i)
print('Costo total=',rutas[1])

Costo    Ruta
[4973.2, 0, 1, 0]
[3346.0, 0, 2, 0]
[6385.299999999999, 0, 3, 0]
[4332.4, 0, 4, 0]
[3204.4, 0, 5, 0]
[2609.8, 0, 6, 0]
[4988.5, 0, 7, 0]
[4843.6, 0, 8, 0]
[5003.5, 0, 9, 0]
Costo total= 39686.700000000004


#### Método de los ahorros - Clarke and Wright

In [13]:
#def clarke_and_wright(Costos, Demandas, Capacidad):
    #paso 1 Seleccionar nodo deposito (0)
    #depot = 0
    #paso 2 Para cada par i-j calcular S_ij (codigo ayudante)
    #paso 3 Ordenar arcos de forma decreciente sin formar subtours (ayudante)

    # Les dejamos una ruta base
    #Rutas = [[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]

    # Escriba su código aquí

    #return Rutas

In [164]:
def clarke_and_wright(costos, demandas, capacidad):

    # Inicializamos las rutas como una lista vacía
    rutas = []

    # Costos es una matriz cuadrada de n x n (siendo n el número de clientes)
    # Calculamos cuantos clientes tiene la instancia
    n = len(costos)

    # s: lista de ahorros (savings)
    ahorros = dict()
    for i in range(n):
        for j in range(n):
            if i!=j and i!=0 and j!=0:
                arc = (i, j)
                savings = round(costos[i][0] + costos[0][j] - costos[i][j], 1)
                ahorros[arc] = savings

    ahorros = dict(sorted(ahorros.items(), key=lambda item: item[1], reverse = True))

    carga_arboles = []

    while len(ahorros) > 0:
        arco_seleccionado = next(iter(ahorros.items()))[0] # llave (i, j) del primer elemento de "ahorros"
        tail = arco_seleccionado[0] # cola del arco
        head = arco_seleccionado[1] # cabeza del arco

        del ahorros[(tail, head)]

        if len(rutas) == 0:
            if demandas[tail] + demandas[head] <= capacidad:
                rutas.append([tail, head])
                carga_arboles.append(demandas[tail] + demandas[head])
                ahorros = {(i, j): value for (i, j), value in ahorros.items() if i != tail}
                ahorros = {(i, j): value for (i, j), value in ahorros.items() if j != head}
                if (head, tail) in list(ahorros.keys()):
                    del ahorros[(head, tail)]
            else:
                rutas.append([tail])
                rutas.append([head])
                carga_arboles.append(demandas[tail])
                carga_arboles.append(demandas[head])

        else:
            head_in_ruta = False
            tail_in_ruta = False
            for ruta in range(len(rutas)):
                if head in rutas[ruta]:
                    head_in_arbol_x = ruta
                    head_in_ruta = True
                if tail in rutas[ruta]:
                    tail_in_arbol_x = ruta
                    tail_in_ruta = True

            entro = False
            for ruta in range(len(rutas)):
                if head == rutas[ruta][0] and carga_arboles[ruta] + demandas[tail] <= capacidad and tail_in_ruta == False:
                    entro = True
                    head_in_ruta = True
                    tail_in_ruta = True
                    rutas[ruta].insert(0, tail)
                    carga_arboles[ruta] += demandas[tail]
                    ahorros = {(i, j): value for (i, j), value in ahorros.items() if i != tail}
                    ahorros = {(i, j): value for (i, j), value in ahorros.items() if j != head}
                    if (head, tail) in list(ahorros.keys()):
                        del ahorros[(head, tail)]
                    break
                elif tail == rutas[ruta][-1] and carga_arboles[ruta] + demandas[head] <= capacidad and head_in_ruta == False:
                    entro = True
                    head_in_ruta = True
                    tail_in_ruta = True
                    rutas[ruta].append(head)
                    carga_arboles[ruta] += demandas[head]
                    ahorros = {(i, j): value for (i, j), value in ahorros.items() if i != tail}
                    ahorros = {(i, j): value for (i, j), value in ahorros.items() if j != head}
                    if (head, tail) in list(ahorros.keys()):
                        del ahorros[(head, tail)]
                    break

            if entro == False:

                if tail_in_ruta == True and head_in_ruta == True and carga_arboles[head_in_arbol_x] + carga_arboles[tail_in_arbol_x] <= capacidad and head_in_arbol_x != tail_in_arbol_x:
                    ruta_combinada = rutas[tail_in_arbol_x] + rutas[head_in_arbol_x]
                    rutas.append(ruta_combinada)
                    rutas[head_in_arbol_x] = [-1]
                    rutas[tail_in_arbol_x] = [-1]
                    carga_arboles.append(carga_arboles[head_in_arbol_x] + carga_arboles[tail_in_arbol_x])
                    carga_arboles[head_in_arbol_x] = -1
                    carga_arboles[tail_in_arbol_x] = -1

                elif tail_in_ruta == True and head_in_ruta == False:
                    rutas.append([head])
                    carga_arboles.append(demandas[head])
                elif tail_in_ruta == False and head_in_ruta == True:
                    rutas.append([tail])
                    carga_arboles.append(demandas[tail])
                elif tail_in_ruta == False and head_in_ruta == False:
                    if demandas[tail] + demandas[head] <= capacidad:
                        rutas.append([tail, head])
                        carga_arboles.append(demandas[tail] + demandas[head])
                        ahorros = {(i, j): value for (i, j), value in ahorros.items() if i != tail}
                        ahorros = {(i, j): value for (i, j), value in ahorros.items() if j != head}
                        if (head, tail) in list(ahorros.keys()):
                            del ahorros[(head, tail)]
                    else:
                        rutas.append([tail])
                        rutas.append([head])
                        carga_arboles.append(demandas[tail])
                        carga_arboles.append(demandas[head])

    while [-1] in rutas:
        rutas.remove([-1])

    while -1 in carga_arboles:
        carga_arboles.remove(-1)

    for ruta in rutas:
        ruta.insert(0, 0)
        ruta.append(0)

    return(rutas)






In [165]:
sol_cw = clarke_and_wright(costos, demandas, 5)
print(sol_cw)

[[0, 9, 0], [0, 1, 8, 0], [0, 5, 0], [0, 6, 0], [0, 7, 3, 4, 2, 0]]


## Parte I-3: Búsquedas locales

#### $\lambda$ interchange 1-0

In [16]:
def lambda_interchange_1_0(Rutas, Costos, Demandas, Capacidad):

    # Escriba su código aquí

    return Rutas

#### $\lambda$ interchange 0-1

In [17]:
def lambda_interchange_1_1(Rutas, Costos, Demandas, Capacidad):

    # Escriba su código aquí

    return Rutas

#### Óptimalidad local

In [18]:
def optimalidad_local(Rutas, Costos, Demandas, Capacidad, Funcion):

    # Escriba su código aquí

    return Rutas

## Parte I-4: Visualización del mapa interactivo

In [19]:
# Usemos la solución obtenida con CW

solucion = clarke_and_wright(costos, demandas, capacidad)

# Para cada ruta
for r in solucion:
    # Reviso todos los arcos y busco el camino entre los pares de nodos
    for index_n in range(len(r)-1):
        camino = caminos[r[index_n], r[index_n+1]]
        C_G = G.subgraph(camino) # Generamos un subgrafo en netowrkx para que se pueda visualizar
        ox.folium.plot_graph_folium(C_G, graph_map=m, tiles='OpenStreetMap', color = 'red', weight= 2)

TypeError: clarke_and_wright() takes 2 positional arguments but 3 were given

In [ ]:
display(m)

## Parte II-1 Calibración de la búsqueda local

In [ ]:
def lambda_interchange_calibrado(Rutas, Costos, Demandas, Capacidad):

    # Escriba su código aquí

    return Rutas

## Parte II-2: Solución del set de instancias

#### Solución un cliente, un vehículo

In [ ]:
solucion_1c1v = un_cliente_un_vehiculo(costos, demandas, capacidad)
print(solucion_1c1v)
print(calcular_costos(solucion_1c1v, distancias, c_v, c_f))
print(es_factible(solucion_1c1v, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Solución inicial Clarke and Wright

In [ ]:
solucion_cw = clarke_and_wright(costos, demandas, capacidad)
print(solucion_cw)
print(calcular_costos(solucion_cw, distancias, c_v, c_f))
print(es_factible(solucion_cw, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Solución un cliente, un vehículo + $\lambda$ interchange 1-0

In [ ]:
solucion_1c1v_lambda01 = optimalidad_local(solucion_1c1v, costos, demandas, capacidad, lambda_interchange_1_0)
print(solucion_1c1v_lambda01)
print(calcular_costos(solucion_1c1v_lambda01, distancias, c_v, c_f))
print(es_factible(solucion_1c1v_lambda01, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Solución un cliente, un vehículo + $\lambda$ interchange 1-1

In [ ]:
solucion_1c1v_lambda11 = optimalidad_local(solucion_1c1v, costos, demandas, capacidad, lambda_interchange_1_1)
print(solucion_1c1v_lambda11)
print(calcular_costos(solucion_1c1v_lambda11, distancias, c_v, c_f))
print(es_factible(solucion_1c1v_lambda11, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Solución C&W + $\lambda$ interchange 1-0

In [ ]:
solucion_cw_lambda01 = optimalidad_local(solucion_cw, costos, demandas, capacidad, lambda_interchange_1_0)
print(solucion_cw_lambda01)
print(calcular_costos(solucion_cw_lambda01, distancias, c_v, c_f))
print(es_factible(solucion_cw_lambda01, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Solución C&W + $\lambda$ interchange 1-1

In [ ]:
solucion_cw_lambda11 = optimalidad_local(solucion_cw, costos, demandas, capacidad, lambda_interchange_1_1)
print(solucion_cw_lambda11)
print(calcular_costos(solucion_cw_lambda11, distancias, c_v, c_f))
print(es_factible(solucion_cw_lambda11, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Soluciones híbridas $\lambda$ interchange 1-0 y 1-1

In [ ]:
solucion_cw_lambda_calibrado = lambda_interchange_calibrado(solucion_cw, costos, demandas, capacidad)
print(solucion_cw_lambda_calibrado)
print(calcular_costos(solucion_cw_lambda_calibrado, distancias, c_v, c_f))
print(es_factible(solucion_cw_lambda_calibrado, demandas, capacidad, n))

[[0, 1, 2, 0], [0, 3, 4, 0], [0, 5, 0], [0, 6, 7, 0], [0, 8, 0], [0, 9, 0]]
(30083.4, 24083.4, 6000)
La ruta es factible.


#### Puede ser útil: leer todos los archivos desde la carpeta 'Instancias' directamente

In [ ]:
# Todos los archivos de la carpeta
files = os.listdir('Instancias')

# Iterar sobre cada archivo de la carpeta
for file_name in files:
     with open('Instancias\\' + str(file_name), 'rb') as file:
        coordenadas = pickle.load(file)
        caminos = pickle.load(file)
        distancias = pickle.load(file)
        demandas = pickle.load(file)
        capacidad = pickle.load(file)
        c_v = pickle.load(file)
        c_f = pickle.load(file)

        # En este caso aplicamos la heurística C&W seguida de lambda interchange 1-1 hasta alcanzar la optimalidad local
        sol = clarke_and_wright(distancias, demandas, capacidad)
        sol = optimalidad_local(sol, distancias, demandas, capacidad, lambda_interchange_1_1)
        costos = calcular_costos(sol, distancias, c_v, c_f)

        print(file_name.split('.')[0], "Costos totales: ", costos[0], "Costos variables: ", costos[1], "Costos fijos: ", costos[2] , es_factible(sol, demandas, capacidad, n = len(demandas)), "La ruta es: ", sol)

#### Nota: al momento de ejecutar esta celda con el archivo tal cual se entrega, es esperable que exista un error de infactibilidad,
        #### pues la solución base entregada es para la instancia con 10 nodos.

PermissionError: [Errno 13] Permission denied: 'Instancias\\Instancias'

## Parte II-3 Ilustración de la búsqueda local

In [ ]:
# Escriba su código aquí